In [1]:
from azureml.core import Workspace

print("Accessing the workspace from job...")
ws = Workspace.from_config("./config")

from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="MyEnvDepl")

print("Creating dependencies...")
myenv_dep = CondaDependencies.create(conda_packages=['scikit-learn', 'pip', 'pandas'],
                                     pip_packages=['azureml-defaults'])

myenv.python.conda_dependencies = myenv_dep

print("Registering the environment...")
myenv.register(ws)

Accessing the workspace from job...
Creating dependencies...
Registering the environment...


{
    "assetId": "azureml://locations/westus/workspaces/54b058d1-e724-4986-aece-5eb3256cb2e6/environments/MyEnvDepl/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240304.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "MyEnvDepl",
    "python": {


In [3]:
from azureml.core.compute import AksCompute

cluster_name = "aks-cluster-001"

aks_config = AksCompute.provisioning_configuration(location="eastus",
                                                   vm_size="STANDARD_D11_V2",
                                                   agent_count=1,
                                                   cluster_purpose="DevTest")

from azureml.core.compute import ComputeTarget
production_cluster = ComputeTarget.create(ws, cluster_name, aks_config)
production_cluster.wait_for_completion(show_output=True)

KeyboardInterrupt: 

In [5]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(environment=myenv,
                                   entry_script="scoring_script.py",
                                   source_directory="./source_files")

from azureml.core.webservice import AksWebservice

deploy_config = AksWebservice.deploy_configuration(cpu_cores=1,
                                                   memory_gb=1)

In [ ]:
from azureml.core.model import Model

model = ws.models['AdultIncome_models']

service = Model.deploy(workspace=ws,
                       name="adultincome-service",
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deploy_config,
                       deployment_target=production_cluster)

service.wait_for_deployment(show_output=True)